In [257]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from datetime import date, timedelta
from datetime import datetime
import time
import urllib.parse
import pandas as pd
import datetime
from bs4 import BeautifulSoup
import re


In [258]:
driver = webdriver.Chrome()
url = "https://x.com"
driver.get(url)
# time.sleep(2)

In [259]:
token_value = "587379e47d4c07095a873be1144effb0f202c589"
cookie = {
    "name" : "auth_token",
    "value" : token_value,
    "domain" : ".x.com"
}
driver.add_cookie(cookie)
driver.refresh()
time.sleep(2)

In [260]:
today = date.today()
yesterday = today - timedelta(days=1)
today = today.strftime("%Y-%m-%d")
yesterday = yesterday.strftime("%Y-%m-%d")
search_term = "pendidikan indonesia"
since_date = yesterday
until_date = today
search_url = f"https://x.com/search?q={search_term}%20since%3A{since_date}%20until%3A{until_date}&src=typed_query"

encoded_search_term = urllib.parse.quote(search_term)
search_url = f"https://x.com/search?q={encoded_search_term}%20since%3A{since_date}%20until%3A{until_date}&src=typed_query"

driver.get(search_url)
time.sleep(5)

In [261]:
# Ambil konten halaman dengan BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')

# List kosong untuk menyimpan data
data = []
scroll_count = 0
max_tweets = 1000

# Loop untuk scroll dan pengambilan data
while len(data) < max_tweets and scroll_count < 10:
    # Ambil konten halaman dengan BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Loop melalui elemen yang sesuai untuk mengambil data
    for div in soup.find_all("div", {"class": "css-175oi2r"}):  # Sesuaikan class sesuai struktur halaman
        heading = div.find("h1").get_text() if div.find("h1") else None
        tweets = div.find_all("article", {"data-testid": "tweet"})
        
        for tweet in tweets:
            tweet_text = tweet.find("div", {"data-testid": "tweetText"}).get_text() if tweet.find("div", {"data-testid": "tweetText"}) else None
            username = (
                tweet.find("div", {"data-testid": "User-Name"})
                .find("span", text=lambda x: x and '@' in x).text
                if tweet.find("div", {"data-testid": "User-Name"}) and tweet.find("div", {"data-testid": "User-Name"}).find("span", text=lambda x: x and '@' in x)
                else ""
            )
            retweets = (
                tweet.find("button", {"data-testid": "retweet"})
                .find("span", {"class": "css-1jxf684"}).text
                if tweet.find("button", {"data-testid": "retweet"})
                else "0"
            )
            likes = (
                tweet.find("button", {"data-testid": "like"})
                .find("span", {"class": "css-1jxf684"}).text
                if tweet.find("button", {"data-testid": "like"})
                else "0"
            )
            views = (
                tweet.find("a", {"aria-label": lambda x: x and 'views' in x})
                .find("span", {"class": "css-1jxf684"}).text
                if tweet.find("a", {"aria-label": lambda x: x and 'views' in x})
                else "0"
            )
            time_element = soup.find("time")
            datetime_value = time_element["datetime"] if time_element else None

            # photo_links = soup.find_all("a", href=re.compile(r"/photo/"))
            # num_photos = len(photo_links)

            # Cari dan ambil status_id dari href dalam tag <a> di setiap tweet
            status_links = tweet.find_all("a", href=re.compile(r"/status/\d+"))
            status_id = None
            for link in status_links:
                match = re.search(r"/status/(\d+)", link['href'])
                if match:
                    status_id = match.group(1)
                    break

            # Tambahkan data ke list
            if len(data) < max_tweets:
                data.append({
                    "Tweet Text": tweet_text,
                    "Username": username,
                    "Likes": likes,
                    "Retweets": retweets,
                    "Views": views,
                    "Status ID": status_id,
                    "Datetime" : datetime_value,
                    # "Photos": num_photos
                })

    # Lakukan scroll untuk memuat tweet berikutnya
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # Tambahkan waktu tunggu jika diperlukan untuk pemuatan halaman
    scroll_count += 1

# Tutup WebDriver
driver.quit()

# Simpan data dalam DataFrame
df = pd.DataFrame(data)





/tmp/ipykernel_9956/2762991448.py:24: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  if tweet.find("div", {"data-testid": "User-Name"}) and tweet.find("div", {"data-testid": "User-Name"}).find("span", text=lambda x: x and '@' in x)
/tmp/ipykernel_9956/2762991448.py:23: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  .find("span", text=lambda x: x and '@' in x).text


In [262]:
df

,Tweet Text,Username,Likes,Retweets,Views,Status ID,Datetime
0,Sektor Pekerjaan Diceburi Warga Indonesia:\n1-...,@LVGIF,4,5,2K,1856170304877560204,2024-11-12T03:01:01.000Z
1,"Dulu Wapres Adam Malik dijuluki ""kancil"". Tern...",@ragilnugroho1,326,55,41K,1856183338740175110,2024-11-12T03:01:01.000Z
2,[bakor] halo! Aku udah tinggal dii korea lbh d...,@BakorFess,861,126,72K,1856466213423198669,2024-11-12T03:01:01.000Z
3,Pemerintah Republik Indonesia meluncurkan 8 Pr...,@bidhumas_jatim,101,72,3.4K,1856211690209329471,2024-11-12T03:01:01.000Z
4,"Beasiswa FULL + Biaya Hidup, Tiket PP & Akomod...",@BeasiswaIndo,1,1,1.3K,1856318269046829326,2024-11-12T03:01:01.000Z
...,...,...,...,...,...,...,...
995,- Teknik penghayatan/imersi bahasa\n\nDi beber...,@EzEnglishWN,,,9,1856359098868216147,2024-11-12T13:08:19.000Z
996,Permasalahan pendidikan yg sering mencuat: zon...,@merseysideliv,3,,219,1856158680057827732,2024-11-12T13:08:19.000Z
997,Seberapa pentingkah peran guru dalam mengubah ...,@inilahdotcom,1,,170,1856234501275562473,2024-11-12T13:08:19.000Z
998,Kayaknya ia terpana liat India bang...\n\nKala...,@limnistik,1,,46,1856240102344929452,2024-11-12T13:08:19.000Z


In [263]:
# Simpan DataFrame ke CSV
csv_file_name = f"tweets_{search_term}_since-{since_date}_until-{until_date}.csv"
df.to_csv("../data-output/"+csv_file_name, index=False)

print(f"Data berhasil disimpan dalam file CSV: {csv_file_name}")

Data berhasil disimpan dalam file CSV: tweets_pendidikan indonesia_since-2024-11-12_until-2024-11-13.csv
